# 0 List all index

In [129]:
!curl -X GET "localhost:9200/_cat/indices?v"

health status index  uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   news   pUcnw5GGQAaeEr00fLQuSg   1   1      20417            0     35.2mb         35.2mb
yellow open   novels cUrnZJIzRKaFFh3AIiOfEw   1   1         33            0     18.3mb         18.3mb
yellow open   test   m_GdMZOqTxesTyykLXxeOA   1   1          3            0      4.2kb          4.2kb


# 1 Delete all indeces

In [2]:
!curl -X DELETE "localhost:9200/*"

{"acknowledged":true}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    21  100    21    0     0     21      0  0:00:01 --:--:--  0:00:01    42


# 2 Index reloaded

In [5]:
%run IndexFilesPreprocess.py --index news  --path 20_newsgroups/**/[0-9]* --token letter --filter lowercase asciifolding

Indexing 20417 files
Reading files ...


DELETE http://localhost:9200/news [status:404 request:0.055s]


Index settings= {'news': {'settings': {'index': {'number_of_shards': '1', 'provided_name': 'news', 'creation_date': '1539639196314', 'analysis': {'analyzer': {'default': {'filter': ['lowercase', 'asciifolding'], 'type': 'custom', 'tokenizer': 'letter'}}}, 'number_of_replicas': '1', 'uuid': 'Y7kUjQfeTxiPH-5w_TdfRw', 'version': {'created': '6040299'}}}}}
Indexing ...


what word is the most frequent one in the English language?

the

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.exceptions import NotFoundError

def count_words(index, alpha):
    lpal = []
    voc = {}
    
    try:
        client = Elasticsearch()

        sc = scan(client, index=index, doc_type='document', query={"query" : {"match_all": {}}})
            
        for s in sc:
            tv = client.termvectors(index=index, doc_type='document', id=s['_id'], fields=['text'])
            if 'text' in tv['term_vectors']:
                for t in tv['term_vectors']['text']['terms']:
                    if t in voc:
                        voc[t] += tv['term_vectors']['text']['terms'][t]['term_freq']
                    else:
                        voc[t] = tv['term_vectors']['text']['terms'][t]['term_freq']
        for v in voc:
            lpal.append((v.encode("utf8", "ignore"), voc[v]))

        print('%s Words' % len(lpal))
    except NotFoundError:
        print('Index %s does not exists' % index)
    
    lpal.sort(key=lambda x: x[0 if alpha else 1])
    
    return lpal

In [4]:
lpal = count_words("novels", False)

54365 Words


In [5]:
lpal[-1]

(b'the', 206706)

# 3 tf-idf and cos similarity

The implementation is in code

## 3.1 Experimenting

First we need to reindex the file, using letter tokenizer and remove stop words in order to improve the performance.


In [174]:
!curl -X DELETE "localhost:9200/*"
%run IndexFilesPreprocess.py --index news  --path 20_newsgroups/**/[0-9]* --token letter --filter lowercase asciifolding stop
%run IndexFilesPreprocess.py --index novels --path novels/*.txt --token letter --filter lowercase asciifolding stop

{"acknowledged":true}Indexing 20417 files
Reading files ...


Index settings= {'news': {'settings': {'index': {'number_of_shards': '1', 'provided_name': 'news', 'creation_date': '1539779959227', 'analysis': {'analyzer': {'default': {'filter': ['lowercase', 'asciifolding', 'stop'], 'type': 'custom', 'tokenizer': 'letter'}}}, 'number_of_replicas': '1', 'uuid': 'CxkdIHXcQ4qWh0hYV_WHmQ', 'version': {'created': '6020499'}}}}}
Indexing ...
Indexing 33 files
Reading files ...


Index settings= {'novels': {'settings': {'index': {'number_of_shards': '1', 'provided_name': 'novels', 'creation_date': '1539779967189', 'analysis': {'analyzer': {'default': {'filter': ['lowercase', 'asciifolding', 'stop'], 'type': 'custom', 'tokenizer': 'letter'}}}, 'number_of_replicas': '1', 'uuid': 'Fq8ez2JHQAu3Ywb2Kmi93w', 'version': {'created': '6020499'}}}}}
Indexing ...


### Test between one document to another. And test one document to itself.

In [175]:
%run TFIDFViewer.py --index novels --files novels/DickensAChristmasCarol.txt novels/DickensGreatExpectations.txt
%run TFIDFViewer.py --index novels --files novels/DickensAChristmasCarol.txt novels/DickensAChristmasCarol.txt

Similarity = 0.01715
Similarity = 1.00000


A simple test we created with two document.

* d1: a b c d e
* d2: a c e f g
* d3: b g f e c

By hand, the similarity between d2 and d3 should be 66%.

In [176]:
%run IndexFilesPreprocess.py --index test  --path mytest/* --token letter

Indexing 3 files
Reading files ...
Index settings= {'test': {'settings': {'index': {'number_of_shards': '1', 'provided_name': 'test', 'creation_date': '1539779983791', 'analysis': {'analyzer': {'default': {'filter': ['lowercase'], 'type': 'custom', 'tokenizer': 'letter'}}}, 'number_of_replicas': '1', 'uuid': 'dOtR6wGmQ2KEqFJ-9RS9WA', 'version': {'created': '6020499'}}}}}
Indexing ...


In [178]:
%run TFIDFViewer.py --index test --print --files mytest/file2.txt mytest/file3.txt

TFIDF FILE mytest/file2.txt
a 0.5773502691896258
f 0.5773502691896258
g 0.5773502691896258
---------------------
TFIDF FILE mytest/file3.txt
b 0.5773502691896258
f 0.5773502691896258
g 0.5773502691896258
---------------------
Similarity = 0.66667


## A final question, have you noticed that we are searching the documents using the path name? 

Yes. We are using path as index to search the document.

## Was the path tokenized by the index? 

No. We configure the path field to prevent the tokenization.

## What did we do differently when indexing the documents so we can look for an exact match in the path field?

We add a property to the mapping of configuration of index.
```python
    client.indices.put_mapping(doc_type='document', index=index, body= {
        "document" : {
            "properties": {
                "path": {
                    "type": "keyword",
                }
            }
        }
    })
```

# 4 Document relevance

In [179]:
%run SearchIndexWeight.py --index news --nhits 5 --query toronto nyc

['toronto', 'nyc']
ID= Sg0JgmYB8gxA2a8bmvTm SCORE=9.764527
PATH= 20_newsgroups/alt.atheism/0000574
TEXT: In article <1r1mr8$eov@aurora.engr.LaTech.edu>, ra
-----------------------------------------------------------------
ID= mQ4JgmYB8gxA2a8bpiUO SCORE=4.2658362
PATH= 20_newsgroups/sci.med/0013128
TEXT: Here is a press release from the Natural Resources
-----------------------------------------------------------------
ID= 4A4JgmYB8gxA2a8brTt0 SCORE=3.966504
PATH= 20_newsgroups/talk.politics.misc/0018667
TEXT: v140pxgt@ubvmsb.cc.buffalo.edu (Daniel B Case) wri
-----------------------------------------------------------------
ID= 1A4JgmYB8gxA2a8bqTDh SCORE=3.5755038
PATH= 20_newsgroups/talk.politics.guns/0015998
TEXT: Jim De Arras (jmd@cube.handheld.com) wrote:
: > La
-----------------------------------------------------------------
4 Documents


In [180]:
%run SearchIndexWeight.py --index news --nhits 5 --query toronto^2 nyc

['toronto^2', 'nyc']
ID= Sg0JgmYB8gxA2a8bmvTm SCORE=13.719839
PATH= 20_newsgroups/alt.atheism/0000574
TEXT: In article <1r1mr8$eov@aurora.engr.LaTech.edu>, ra
-----------------------------------------------------------------
ID= 4A4JgmYB8gxA2a8brTt0 SCORE=6.641708
PATH= 20_newsgroups/talk.politics.misc/0018667
TEXT: v140pxgt@ubvmsb.cc.buffalo.edu (Daniel B Case) wri
-----------------------------------------------------------------
ID= mQ4JgmYB8gxA2a8bpiUO SCORE=5.9937963
PATH= 20_newsgroups/sci.med/0013128
TEXT: Here is a press release from the Natural Resources
-----------------------------------------------------------------
ID= 1A4JgmYB8gxA2a8bqTDh SCORE=5.0238314
PATH= 20_newsgroups/talk.politics.guns/0015998
TEXT: Jim De Arras (jmd@cube.handheld.com) wrote:
: > La
-----------------------------------------------------------------
4 Documents


In [181]:
%run SearchIndexWeight.py --index news --nhits 5 --query toronto nyc^2

['toronto', 'nyc^2']
ID= Sg0JgmYB8gxA2a8bmvTm SCORE=15.573743
PATH= 20_newsgroups/alt.atheism/0000574
TEXT: In article <1r1mr8$eov@aurora.engr.LaTech.edu>, ra
-----------------------------------------------------------------
ID= mQ4JgmYB8gxA2a8bpiUO SCORE=6.803713
PATH= 20_newsgroups/sci.med/0013128
TEXT: Here is a press release from the Natural Resources
-----------------------------------------------------------------
ID= 1A4JgmYB8gxA2a8bqTDh SCORE=5.70268
PATH= 20_newsgroups/talk.politics.guns/0015998
TEXT: Jim De Arras (jmd@cube.handheld.com) wrote:
: > La
-----------------------------------------------------------------
ID= 4A4JgmYB8gxA2a8brTt0 SCORE=5.257805
PATH= 20_newsgroups/talk.politics.misc/0018667
TEXT: v140pxgt@ubvmsb.cc.buffalo.edu (Daniel B Case) wri
-----------------------------------------------------------------
4 Documents


# 5 Rocchio

The detail about of implementation is in the code.

```bash
usage: Rocchio.py [-h] --index INDEX [--k K] [--nrounds NROUNDS] [--R R]
                  [--alpha ALPHA] [--beta BETA] [--query ...]
```


In [182]:
%run Rocchio.py --k 5 --nrounds 5 --index news --query nice city

Receive query ['nice', 'city']
******************************************************
iteration 1
Term vector : ['nice', 'city']
Weight vector : [1.0, 1.0]
Composed query : ['nice^1.0', 'city^1.0']
******************************************************
iteration 2
Term vector : ['nice', 'city']
Weight vector : [1.0611689247450002, 1.1029373383751016]
Composed query : ['nice^1.0611689247450002', 'city^1.1029373383751016']
******************************************************
iteration 3
Term vector : ['nice', 'city']
Weight vector : [1.1223378494900005, 1.205874676750203]
Composed query : ['nice^1.1223378494900005', 'city^1.205874676750203']
******************************************************
iteration 4
Term vector : ['nice', 'city']
Weight vector : [1.1835067742350007, 1.3088120151253047]
Composed query : ['nice^1.1835067742350007', 'city^1.3088120151253047']
******************************************************
iteration 5
Term vector : ['nice', 'city']
Weight vector : [1.244675

## 5.1 Experimenting

### Do the queries that pseudorelevance feedback produce make sense? for example, do the new terms seem related to what the user is looking for?

In certain point, yes. We can the previous query nice city. Nice is a adjective, it has less meaning. And City should get more weight. So it makes sense.

### In which sense do the results improve? Recall? Precision?

Since in this experiment we assume always first k documents are relevant. So we can't say either of recall or precision has been improved. However, in common sense, if we emphasize some key terms, then the retrieved documents have higher possibility to be relevant. So recall and precision both would be improved. Specially recall in first rounds. And then the precision will be improved because the ranking will be changed.

### Investigate to some extent the effect of each parameter. 

#### default query

In [205]:
%run Rocchio.py --k 10 --nrounds 1 --R 1 --alpha 1 --beta 1 --index news --query nice city^0.5

Receive query ['nice', 'city^0.5']
******************************************************
iteration 1
Term vector : ['nice', 'city']
Weight vector : [1.0, 0.5]
Composed query : ['nice^1.0', 'city^0.5']
******************************************************
ID= CQ4JgmYB8gxA2a8brj4t SCORE=7.811672
PATH= 20_newsgroups/talk.religion.misc/0019033
-----------------------------------------------------------------
ID= 8A4JgmYB8gxA2a8bohYp SCORE=6.3309307
PATH= 20_newsgroups/rec.sport.baseball/0009383
-----------------------------------------------------------------
ID= bA4JgmYB8gxA2a8bohUp SCORE=6.239165
PATH= 20_newsgroups/rec.motorcycles/0008995
-----------------------------------------------------------------
ID= ww4JgmYB8gxA2a8bnwri SCORE=5.929438
PATH= 20_newsgroups/misc.forsale/0006276
-----------------------------------------------------------------
ID= oA4JgmYB8gxA2a8bohYp SCORE=5.688919
PATH= 20_newsgroups/rec.sport.baseball/0009303
----------------------------------------------------

#### change nrounds and R

In [206]:
%run Rocchio.py --k 10 --nrounds 10 --R 100 --alpha 1 --beta 1 --index news --query nice city^0.5

Receive query ['nice', 'city^0.5']
******************************************************
iteration 1
Term vector : ['nice', 'city']
Weight vector : [1.0, 0.5]
Composed query : ['nice^1.0', 'city^0.5']
******************************************************
iteration 2
Term vector : ['nice', 'city']
Weight vector : [1.0434602376905573, 0.5821450261671961]
Composed query : ['nice^1.0434602376905573', 'city^0.5821450261671961']
******************************************************
iteration 3
Term vector : ['nice', 'city']
Weight vector : [1.0869204753811146, 0.6642900523343922]
Composed query : ['nice^1.0869204753811146', 'city^0.6642900523343922']
******************************************************
iteration 4
Term vector : ['nice', 'city']
Weight vector : [1.1303807130716719, 0.7464350785015883]
Composed query : ['nice^1.1303807130716719', 'city^0.7464350785015883']
******************************************************
iteration 5
Term vector : ['nice', 'city']
Weight vector : [1.

In [207]:
%run Rocchio.py --k 10 --nrounds 50 --R 100 --alpha 1 --beta 1 --index news --query nice city^0.5

Receive query ['nice', 'city^0.5']
******************************************************
iteration 1
Term vector : ['nice', 'city']
Weight vector : [1.0, 0.5]
Composed query : ['nice^1.0', 'city^0.5']
******************************************************
iteration 2
Term vector : ['nice', 'city']
Weight vector : [1.0434602376905573, 0.5821450261671961]
Composed query : ['nice^1.0434602376905573', 'city^0.5821450261671961']
******************************************************
iteration 3
Term vector : ['nice', 'city']
Weight vector : [1.0869204753811146, 0.6642900523343922]
Composed query : ['nice^1.0869204753811146', 'city^0.6642900523343922']
******************************************************
iteration 4
Term vector : ['nice', 'city']
Weight vector : [1.1303807130716719, 0.7464350785015883]
Composed query : ['nice^1.1303807130716719', 'city^0.7464350785015883']
******************************************************
iteration 5
Term vector : ['nice', 'city']
Weight vector : [1.

******************************************************
iteration 39
Term vector : ['nice', 'city']
Weight vector : [2.651489032241174, 3.761091493181384]
Composed query : ['nice^2.651489032241174', 'city^3.761091493181384']
******************************************************
iteration 40
Term vector : ['nice', 'city']
Weight vector : [2.694949269931731, 3.851960300525326]
Composed query : ['nice^2.694949269931731', 'city^3.851960300525326']
******************************************************
iteration 41
Term vector : ['nice', 'city']
Weight vector : [2.738409507622288, 3.942829107869268]
Composed query : ['nice^2.738409507622288', 'city^3.942829107869268']
******************************************************
iteration 42
Term vector : ['nice', 'city']
Weight vector : [2.781869745312845, 4.033697915213209]
Composed query : ['nice^2.781869745312845', 'city^4.033697915213209']
******************************************************
iteration 43
Term vector : ['nice', 'city']
Weigh

#### Change alpha and beta

In [208]:
%run Rocchio.py --k 10 --nrounds 20 --R 100 --alpha 0.95 --beta 1 --index news --query nice city^0.5

Receive query ['nice', 'city^0.5']
******************************************************
iteration 1
Term vector : ['nice', 'city']
Weight vector : [1.0, 0.5]
Composed query : ['nice^1.0', 'city^0.5']
******************************************************
iteration 2
Term vector : ['nice', 'city']
Weight vector : [0.9934602376905572, 0.5571450261671961]
Composed query : ['nice^0.9934602376905572', 'city^0.5571450261671961']
******************************************************
iteration 3
Term vector : ['nice', 'city']
Weight vector : [0.9872474634965867, 0.6114328010260324]
Composed query : ['nice^0.9872474634965867', 'city^0.6114328010260324']
******************************************************
iteration 4
Term vector : ['nice', 'city']
Weight vector : [0.9813453280123144, 0.6630061871419268]
Composed query : ['nice^0.9813453280123144', 'city^0.6630061871419268']
******************************************************
iteration 5
Term vector : ['nice', 'city']
Weight vector : [0.

In [209]:
%run Rocchio.py --k 10 --nrounds 20 --R 100 --alpha 0.5 --beta 0.5 --index news --query nice city^0.5

Receive query ['nice', 'city^0.5']
******************************************************
iteration 1
Term vector : ['nice', 'city']
Weight vector : [1.0, 0.5]
Composed query : ['nice^1.0', 'city^0.5']
******************************************************
iteration 2
Term vector : ['nice', 'city']
Weight vector : [0.5217301188452786, 0.29107251308359805]
Composed query : ['nice^0.5217301188452786', 'city^0.29107251308359805']
******************************************************
iteration 3
Term vector : ['nice', 'city']
Weight vector : [0.2825951782679179, 0.18660876962539708]
Composed query : ['nice^0.2825951782679179', 'city^0.18660876962539708']
******************************************************
iteration 4
Term vector : ['nice', 'city']
Weight vector : [0.16302770797923757, 0.1343768978962966]
Composed query : ['nice^0.16302770797923757', 'city^0.1343768978962966']
******************************************************
iteration 5
Term vector : ['nice', 'city']
Weight vector

### Do you get very different results if you change the parameters nrounds, k, R, α, β? Do you find, for each one, some value or value range that seems to be optimal in some sense?

We can see from previous experiments. We get diffrent results after change those parameters. Specially in case if we have initial query with some term weight very low, but by the Rocchio, it increase and become larger weight than other terms. Then it may perform very different result. Nrounds is the parameter to emphasize the effect of Rocchio, its effect depends on other parameters. Finally we can observe alpha and beta are very important in Rocchio algorithm. If alpha and beta are too low, the weight vector will converge to zero. So a reasonable value for alpha and beta is needed.


By the observation from previous experiments. The program cannot run too slow, nrounds seems reasonable in range of (5,20). R cannot be too large, otherwise it will take into account every words. In this dataset we have more than 10k words. So we first 100 - 500 top weighted words seem to be good. Since we don't have γ, so no penalization is performed in this experiement. Then we need alpha to be a little small than 1 but not too small to be able to reduce the effect from original query, 0.95 is suitable in this case. Finally the beta can be any value depends on how much effect we want to get from the retried relevant documents. It can be any value bigger than decrece rate of alpha, we would set it in 0.8 or 0.9 to adapt the pace of nrounds parameter.

In [211]:
%run Rocchio.py --k 10 --nrounds 20 --R 250 --alpha 0.95 --beta 0.9 --index news --query nice city^0.5

Receive query ['nice', 'city^0.5']
******************************************************
iteration 1
Term vector : ['nice', 'city']
Weight vector : [1.0, 0.5]
Composed query : ['nice^1.0', 'city^0.5']
******************************************************
iteration 2
Term vector : ['nice', 'city']
Weight vector : [0.9943515939572776, 0.5518734354069282]
Composed query : ['nice^0.9943515939572776', 'city^0.5518734354069282']
******************************************************
iteration 3
Term vector : ['nice', 'city']
Weight vector : [0.9889856082166912, 0.6011531990435101]
Composed query : ['nice^0.9889856082166912', 'city^0.6011531990435101']
******************************************************
iteration 4
Term vector : ['nice', 'city']
Weight vector : [0.9838879217631342, 0.6479689744982627]
Composed query : ['nice^0.9838879217631342', 'city^0.6479689744982627']
******************************************************
iteration 5
Term vector : ['nice', 'city']
Weight vector : [0.